# Multiple Quantum Wells and Finite Superlattices

This notebook processes the results generated by next**nano**++ for the simulation of finite superlattices, as outlined in Paul Harrison's book [Quantum Wells, Wires and Dots (2nd e.d)](https://www.wiley.com/en-us/Quantum+Wells%2C+Wires+and+Dots%3A+Theoretical+and+Computational+Physics+of+Semiconductor+Nanostructures%2C+4th+Edition-p-9781118923368). The section refered to is titled "Multiple Quantum Wells and Finite Superlattices" (Section 3.10, pp. 95-96)

In that section, $N$ repeats of 40Å GaAs / 40Å Ga<sub>0.8</sub>Al<sub>0.2</sub>As sandwiched between 200Å Ga<sub>0.8</sub>Al<sub>0.2</sub>As on the left and right. We will designate the coordinate $x=0$ as the start of the first well.

## Python libraries used

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import nextnanopy as nn
import pathlib
import os
import re
%matplotlib inline

## Checking the configuration of nextnanopy

The path directories in the config file must be set up. If they are not, refer to the [tutorial on setting up config files](https://github.com/nextnanopy/nextnanopy/blob/master/docs/examples/Example0_Set_up_the_configuration.ipynb).

In [2]:
software = 'nextnano++'
nn.config.config[software]

{'exe': 'C:\\Program Files\\nextnano\\2021_12_24\\nextnano++\\bin 64bit\\nextnano++_Intel_64bit.exe',
 'license': 'C:\\Users\\brandon.loke\\Documents\\nextnano\\License\\License_nnp.lic',
 'database': 'C:\\Program Files\\nextnano\\2021_12_24\\nextnano++\\Syntax\\database_nnp.in',
 'outputdirectory': 'D:\\nextnano output',
 'threads': 0}

We want to obtain all the paths, so we set up a simple function to aid us.

In [3]:
def get_directories(software):
    try:
        return nn.config.config[software]
    except:
        raise KeyError(f'Software name not recognised. Please choose one of: {list(nn.config.config.keys())}')

exe, license, database, output_folder, threads = list(get_directories('nextnano++').values())

These two functions will be useful later for manipulating files when the sweep variables are iterated over for execution

In [4]:
def remove_extension(filename):
    return re.match(r'([^.]+)',filename).group(0)

def get_extension(filename):
    return '.'+ re.search(r'\.(.*)',filename).group(1)

## Setting up the sweeping variable

We will sweep through the variables using <code>nextnanopy</code>. We define
* The input folder
* The filename
* The variable of interest
* The values to be swept over. In this case, we want to simulate $2\leq N \leq 10$ wells

In [5]:
input_folder = r'D:\nextnano tutorials\1DSuperlattice_N_Wells'
filename = r'Superlattice_N_Wells.in'
SweepVariable = 'NUMBER_OF_WELLS'
SweepValues = np.arange(2,11,1)

## Setting up the folder for results

We will also generate a folder for <code>nextnanopy</code> results.

In [6]:
def generate_python_folder(software):
    output_folder = nn.config.config[software]['outputdirectory']
    folder_output_python = os.path.join(output_folder, r'nextnanopy')
#     print(folder_output_python)
    nn.utils.misc.mkdir_if_not_exist(folder_output_python)
    pass
generate_python_folder(software)

## Getting the input file
We now run a block of code to get the input file for manipulation later.

In [7]:
def get_input_file(input_folder, filename):
    input_path = os.path.join(input_folder, filename)
    return nn.InputFile(input_path)

input_file = get_input_file(input_folder, filename)

In [8]:
input_file.fullpath

'D:\\nextnano tutorials\\1DSuperlattice_N_Wells\\Superlattice_N_Wells.in'

### Alternative way of getting the list of values
In next**nano**++, some variables already have a list of values or a range of values set up. This can be grabbed from the raw input file with the following functions

In [9]:
def grab_list_variables(input_file, VariableName):
    c = input_file.variables[VariableName].comment
    try:
        return np.array(list(map(float, re.search(r'(ListOfValues:)(.*?)\)', c).group(2).split(','))))
    except:
            raise ValueError("No list of values found for variable in input file. Check that variable has ListOfValues in comment or manually create a list of values.")
            
def grab_range_variables(input_file, VariableName):
    c = input_file.variables[VariableName].comment
    try:
        return np.array(list(map(float, re.search(r'(RangeOfValues:)(.*?)\)', c).group(2).split(','))))
    except:
            raise ValueError("No range of values found for variable in input file. Check that variable has RangeOfValues in comment or manually create a list of values.")     

For example, if we wanted to get the default list of values for the <code>$NUMBER_OF_WELLS</code> variable in next**nano**++, we may do the following.

In [10]:
grab_list_variables(input_file, "NUMBER_OF_WELLS")

array([ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

## Simulating batch files
We now have the list of variables we want to sweep over. A function can be defined to iterate over these values and output the wanted files into a data structure. This data structure is a dictonary whose structure mirrors that of the output folder.

In [11]:
'''
DO NOT CHANGE THE KEYS, ONLY CHANGE THE FILES IN THE LIST
Please make sure that the correct subfolder is specified with the current datafile inside, else an
error will be thrown.
'''
datafiles = {
    r'bias_00000' : ['bandedges.dat'],
    r'bias_00000\Quantum' : ['amplitudes_shift_quantum_region_Gamma_00000.dat']
}

def get_filename_no_extension(input_file):
    file = re.search(r'[^\\]*$', input_file.fullpath).group(0)
    return remove_extension(file)

def get_filename_extension(input_file):
    file = re.search(r'[^\\]*$', input_file.fullpath).group(0)
    return get_extension(file)

def sweep_execute_variable(VariableName, ListOfValues, input_file, software, biases, datafiles):
    software = 'nextnano++'
    d = {}
    comment_original = input_file.variables[VariableName].comment
    
    filename_no_extension = get_filename_no_extension(input_file)
    FileExtension = get_filename_extension(input_file)
    
    folder_output = nn.config.config[software]['outputdirectory']
    
    for value in ListOfValues:
        d[str(value)] = {}

        VariableName_Value = VariableName + '_' + str(value)
        input_file.set_variable(VariableName, value = value, comment='<= PYTHON <= ' + comment_original)
        input_filename_variable = filename_no_extension + '_' + VariableName_Value
        my_input_file_new = os.path.join(folder_output, input_filename_variable + FileExtension) #This is currently hardcoded

        input_file.save(my_input_file_new, overwrite = True, automkdir = True)
        input_file.execute()
        
        for bias in biases:
            d[str(value)][bias] = {}
            
            for k,v in datafiles.items():
#                 k = k.replace('00000', bias)
                temp_key = re.search(r'([^\\]+$)', k).group(0)
                if 'bias' in temp_key:
                    d[str(value)][bias] = {}
                
                    for dat in v:
                        dat = dat.replace('00000', bias)
                        p = os.path.join(folder_output, input_filename_variable + fr'\{k}' + fr'\{dat}')
                        data = nn.DataFile(p, product = software)
                        d[str(value)][bias][dat] = data
                else:
                    d[str(value)][bias][temp_key] = {}
                
                    for dat in v:
                        dat = dat.replace('00000', bias)
                        p = os.path.join(folder_output, input_filename_variable + fr'\{k}' + fr'\{dat}')
                        data = nn.DataFile(p, product = software)
                        d[str(value)][bias][temp_key][dat] = data
    return d

d = sweep_execute_variable(SweepVariable, SweepValues, input_file, software, ['00000'], datafiles)

STARTING...
Starting execution as:
C:\Program Files\nextnano\2021_12_24\nextnano++\bin 64bit\nextnano++_Intel_64bit.exe --license C:\Users\brandon.loke\Documents\nextnano\License\License_nnp.lic --database C:\Program Files\nextnano\2021_12_24\nextnano++\Syntax\database_nnp.in --threads 0 --outputdirectory D:\nextnano output\Superlattice_N_Wells_NUMBER_OF_WELLS_2 --noautooutdir D:\nextnano output\Superlattice_N_Wells_NUMBER_OF_WELLS_2.in 

nextnano++ (1.9.3 - 2021.122001) Jan 25 2022
COPYRIGHT NOTICE                                                             
Please read the file 'copyright_nextnano++.txt' in your installation folder  
for further information about the terms of copyright of the nextnano++ code  
and of third party source codes and libraries used in the nextnano++ code.   
                                                                             
In case this file is missing or seems incomplete or corrupted, please contact
nextnano GmbH, Germany by submitting a suppo

  Newton achieved/desired residual: 1.19472476e-12 1.80951265e-10

Solving Quantum Mechanics ---- (and calculate density)

  valence band maximum: -0.794876818
  conduction band minimum: 0.627605325

 Solving 1 approximate (subspace) 1-band Schroedinger equation(s):
   Solving dense hermitian eigenvalue problem (standard solver)...
      Full real symmetric eigenvalue solver:           1          70
      
 Solving 1 approximate (subspace) 1-band Schroedinger equation(s):
   Solving dense hermitian eigenvalue problem (standard solver)...
      Full real symmetric eigenvalue solver:           1         250
      
 Solving 1 approximate (subspace) 1-band Schroedinger equation(s):
   Solving dense hermitian eigenvalue problem (standard solver)...
      Full real symmetric eigenvalue solver:           1          70
      
 Solving 1 approximate (subspace) 1-band Schroedinger equation(s):
   Solving dense hermitian eigenvalue problem (standard solver)...
      Full real symmetric eigenvalue


Material database initialized.

Checking license:
	 Valid From: 2022-1-1 To: 2022-12-31
	 Licensed to: E-mail: brandon.loke@nextnano.com-2022-12-31-de-0000-00-nxt3

********* SETUP SIMULATION *****************************************************

NOTE: Using output directory specified from command line (nextnanomat), 

NOTE: Setting output directory to: D:\nextnano output\Superlattice_N_Wells_NUMBER_OF_WELLS_3\

********* Simulation Grid *********

Creating grid 1 using:
    pos = -21     	spacing = 1
    pos = -20     	spacing = 0.1
    pos = 40     	spacing = 0.1
    pos = 41     	spacing = 0.1

Grid dimension: 612 * 1 * 1 
Number of unique grid points: 612
Range in 1-direction:   -21 , ... , 41


********* Rotation Matrix *********
 1.0000000,  0.0000000,  0.0000000
 0.0000000,  1.0000000,  0.0000000
 0.0000000,  0.0000000,  1.0000000

********* Periodicity *************
  0-direction:  not periodic


Start initializing structure.
Finished initializing structure.
Structure initiali


Computing densities...

QUANTUM-CURRENT-POISSON:   Residual_EDensity = 4.725720942e-13      Residual_HDensity = 7.354396530e-14
QUANTUM-CURRENT-POISSON:   Residual_EFermilevel = 0.000000000e+00   Residual_HFermilevel = 0.000000000e+00
QUANTUM-CURRENT-POISSON:   Residual_Potential = 0.000000000e+00

*----  SOLVING QUANTUM-CURRENT-POISSON EQUATIONS FINISHED ----------------------------

Calculating classical integrated carrier densities as function of energy.

Calculating quantum integrated carrier densities as function of energy.

Calculating classical energy-resolved carrier densities.

Calculating quantum energy-resolved carrier densities.

Solving Quantum Mechanics ---- (quantum regions without density only)


Solving Quantum Mechanics ---- (determine k-dispersion only)

   Calculating interband matrix elements...
   Calculating interband matrix elements...
   Calculating interband matrix elements...
   Calculating (only) transition energies...
   Calculating (only) transition energ

Calculating hole mobility...
Calculating variable recombination and generation...
Calculating fixed generation/recombination/injection...

Solving electron current equation(s)...

Solving hole current equation(s)...

Computing densities...

Current-Density:   Residual_EFermilevel = 0.000000000e+00   Residual_HFermilevel = 0.000000000e+00

Current-repetition: (done)  ---------------------------

Solving nonlinear Poisson equation using predicted quantum densities...
  Newton step: 1	1.541720510200749e-12
  Newton achieved/desired residual: 1.51049037e-12 1.80951265e-10

Solving Quantum Mechanics ---- (and calculate density)

  valence band maximum: -0.794876818
  conduction band minimum: 0.627605325

 Solving 1 approximate (subspace) 1-band Schroedinger equation(s):
   Solving dense hermitian eigenvalue problem (standard solver)...
      Full real symmetric eigenvalue solver:           1          70
      
 Solving 1 approximate (subspace) 1-band Schroedinger equation(s):
   Solving den


Material database initialized.

Checking license:
	 Valid From: 2022-1-1 To: 2022-12-31
	 Licensed to: E-mail: brandon.loke@nextnano.com-2022-12-31-de-0000-00-nxt3

********* SETUP SIMULATION *****************************************************

NOTE: Using output directory specified from command line (nextnanomat), 

NOTE: Setting output directory to: D:\nextnano output\Superlattice_N_Wells_NUMBER_OF_WELLS_5\

********* Simulation Grid *********

Creating grid 1 using:
    pos = -21     	spacing = 1
    pos = -20     	spacing = 0.1
    pos = 56     	spacing = 0.1
    pos = 57     	spacing = 0.1

Grid dimension: 772 * 1 * 1 
Number of unique grid points: 772
Range in 1-direction:   -21 , ... , 57


********* Rotation Matrix *********
 1.0000000,  0.0000000,  0.0000000
 0.0000000,  1.0000000,  0.0000000
 0.0000000,  0.0000000,  1.0000000

********* Periodicity *************
  0-direction:  not periodic


Start initializing structure.
Finished initializing structure.
Structure initiali

  Newton achieved/desired residual: 1.4881637e-12 1.80951265e-10

Solving Quantum Mechanics ---- (and calculate density)

  valence band maximum: -0.794876818
  conduction band minimum: 0.627605325

 Solving 1 exact 1-band Schroedinger equation(s):
    Tridiagonal real symmetric eigenvalue solver:           1          70
      
 Solving 1 exact 1-band Schroedinger equation(s):
    Tridiagonal real symmetric eigenvalue solver:         512         761
      
 Solving 1 exact 1-band Schroedinger equation(s):
    Tridiagonal real symmetric eigenvalue solver:         692         761
      
 Solving 1 exact 1-band Schroedinger equation(s):
    Tridiagonal real symmetric eigenvalue solver:         662         761
      

Computing densities...

QUANTUM-CURRENT-POISSON:   Residual_EDensity = 1.158062927e-12      Residual_HDensity = 6.175286366e-14
QUANTUM-CURRENT-POISSON:   Residual_EFermilevel = 0.000000000e+00   Residual_HFermilevel = 0.000000000e+00
QUANTUM-CURRENT-POISSON:   Residual_Poten

KeyboardInterrupt: 

We can explore the structure of the output file by looking at the keys and exploring.

In [ ]:
d.keys()

The above correspond to the list of values iterated over. Note that they are strings.

In [ ]:
d['2'].keys()

This refers to the value of the bias applied. Since no bias was applied, we only have <code>'00000'</code>

In [ ]:
d['2']['00000']

We see that there is <code>bandedges.dat</code> and a <code>Quantum</code> key. This mirrors the output folder. Suppose we want to obtain the band edge $\Gamma$ energies. This can be done with

In [ ]:
def get_array(data, data_filename, quantity_of_interest, func = lambda x:x, bias = '00000', subfolder = None):
    '''
    Gets a single array from the data structure with the value of interest specified in the argument
    '''
    arr = np.array([])
    if subfolder:
        return np.append(arr, list(map(func, data[bias][subfolder][data_filename].variables[quantity_of_interest].value)))
    return np.append(arr, list(map(func, data[bias][data_filename].variables[quantity_of_interest].value)))

The above is a function to obtain a single array from the data structure. Since <code>bandedges.dat</code> is at the top of the file strucutre (under <code>bias_00000</code>), we specify <code>subfolder = None</code>.

In [ ]:
gamma = get_array(data = d['2'], bias = '00000', subfolder = None, 
                  data_filename = 'bandedges.dat', quantity_of_interest = 'Gamma')
gamma

To make a plot of the $\Gamma$ band edge against the dimensions of the device, we obtain the x-coordinates with

In [ ]:
def get_coords(data, data_filename, direction, func = lambda x:x, bias = '00000', subfolder = None):
    '''
    Gets a single array from the data structure with the value of interest specified in the argument
    '''
    arr = np.array([])
    if subfolder:
        return np.append(arr, list(map(func, data[bias][subfolder][data_filename].coords[direction].value)))
    return np.append(arr, list(map(func, data[bias][data_filename].coords[direction].value)))

In [ ]:
x = get_coords(data = d['2'], data_filename = 'bandedges.dat', direction = 'x', bias = '00000', subfolder = None)

Before we plot the $\Gamma$ energies against $x$, we check that the dimensions are identical

In [ ]:
np.size(x) == np.size(gamma)

## Plotting the $\Gamma$ band edges

In [ ]:
palette = ['#343131','#13adb5']
plt.style.use('default')
plt.style.use('seaborn-deep')

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, gamma)

## Recreating Harrison's Figures
Harrison's figures take the bottom of the quantum well to be at 0 potential. Differences in how next**nano**++ calculates the band edges result in a different offset in the values outputted by the software.

We will first obtain the offset with

In [ ]:
bandedgeoffset = np.amin(gamma[30:-30]) #Values chosen to avoid the 0 value at the start and end
bandedgeoffset

If the offset is known before hand, one can use the <code>func</code> argument in the function <code>get_array</code> to shift all values of energy by that offset amount. This can also be done with <code>numpy</code> array manipulation. Both approaches are equivalent

In [ ]:
#with get_array function
gamma_shift_func = get_array(data = d['2'], bias = '00000', subfolder = None, 
                        data_filename = 'bandedges.dat', quantity_of_interest = 'Gamma',
                       func = lambda x: x -bandedgeoffset)

#numpy array manipulation
gamma_shift_mani = gamma - bandedgeoffset

In [ ]:
np.all(gamma_shift_func == gamma_shift_mani) 
#True indicates that all values obtained from the function match the ones outputted by numpy array manipulation

### Finding groundstate energies

To find the groundstate energies as a function of the number of wells, we will create a simple function to iterate over all the $N$ wells. The groundstate energies are found in the <code>Quantum</code> subfolder

In [ ]:
d['2']['00000']['Quantum']['amplitudes_shift_quantum_region_Gamma_00000.dat'].variables['E_1'].value

In [ ]:
'''
the function below initialises an array
for every value in the list we want, we get the array for minimum energies
We shift it by the offset and multiply by 1000 to get units of meV
We get the minimum values of this array
append value to initialised array
'''
def get_all_energies(listvalues, data):
    arr = np.array([])
    for val in listvalues:
        min_energy = get_array(data = d[str(val)], bias = '00000', subfolder = 'Quantum',
                              data_filename = 'amplitudes_shift_quantum_region_Gamma_00000.dat',
                              quantity_of_interest = 'E_1', func = lambda x : (x - bandedgeoffset)*1000)
        single_value = np.amin(min_energy) #we just need one number, not the entire array
        arr = np.append(arr, single_value)
    return arr

In [ ]:
groundstate_energies = get_all_energies(SweepValues, d)
groundstate_energies

In [ ]:
fig, ax = plt.subplots()
ax.plot(SweepValues, groundstate_energies, 'o--', color = palette[1])
# ax.grid(visible = True)
ax.set_ylabel('Energy (meV)')
ax.set_xlabel('Number of wells')
ax.set_title('Ground state energies')

# plt.savefig('./Images/EnergyVsN.png', dpi = 2000)

### Wavefunction in a finite superlattice
The first case has the following parameters:
* 40Å GaAs / 40Å Ga<sub>0.8</sub>Al<sub>0.2</sub>As sandwiched between 200Å Ga<sub>0.8</sub>Al<sub>0.2</sub>As on the left and right.
* 10 wells
* 4nm quantum wells
* 4nm barriers

**Getting the wavefunction and the x coordinates** <br>
We only want the groundstate wavefunction amplitudes.

Note that we cannot use the x-coordinates found in <code>bandedges.dat</code> for <code>amplitudes_shift_quantum_region_Gamma_00000.dat</code> because the dimensions of the array are not equal.

In [ ]:
wf1 = get_array(data = d['10'], subfolder = 'Quantum', data_filename = 'amplitudes_shift_quantum_region_Gamma_00000.dat', 
                quantity_of_interest = 'Psi_1', func = lambda x: (x - bandedgeoffset)*1000)
bandedge10 = get_array(data = d['10'], data_filename = 'bandedges.dat', 
                quantity_of_interest = 'Gamma', func = lambda x: (x - bandedgeoffset)*1000)
x = get_coords(data = d['10'], data_filename = 'bandedges.dat', direction = 'x', 
               bias = '00000', subfolder = None)

xwf1 = get_coords(data = d['10'], data_filename = 'amplitudes_shift_quantum_region_Gamma_00000.dat',
                 subfolder = 'Quantum', bias = '00000', direction = 'x')

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, bandedge10, color = palette[0])
ax.plot(xwf1, wf1, color = palette[1])

ax.set_ylim(0,300)
ax.set_xlabel('Distance (nm)')
ax.set_ylabel('Energy (meV)')
ax.set_title('Finite Superlattice')

# plt.savefig('./Images/wf1.png', dpi = 2000)

### Wavefunction in a multiple quantum well system

The second case plotted has the following parameters:
* 100Å GaAs / 100Å Ga<sub>0.6</sub>Al<sub>0.4</sub>As sandwiched between 100Å Ga<sub>0.6</sub>Al<sub>0.4</sub>As on the left and right.
* 4 wells
* 10nm quantum wells
* 10nm barriers

Note that Ga<sub>x</sub>Al<sub>1-x</sub>As used here is different from the earlier graph plotted.

The alloy composition and the thickness of the barriers and the quantum wells are different in this example. We will use nextnanopy to change the variables and output the result.

In [ ]:
input_file.fullpath

In [ ]:
input_file.variables.keys()

We want to change
* <code>WELl_WIDTH</code>
* <code>BARRIER_WIDTH</code>
* <code>LEFT_BARRIER_WIDTH</code>
* <code>RIGHT_BARRIER_WIDTH</code>
* <code>NUMBER_OF_WELLS</code>

In addition to this, we will save this new input file into the output folder.

In [ ]:
input_file.set_variable(name = "WELL_WIDTH", value = 10)
input_file.set_variable(name = "BARRIER_WIDTH", value = 10)
input_file.set_variable(name = "LEFT_BARRIER_WIDTH" , value = 10)
input_file.set_variable(name = "RIGHT_BARRIER_WIDTH" , value = 10)
input_file.set_variable(name = "NUMBER_OF_WELLS", value = 4)


my_input_file_new = os.path.join(output_folder, r'Superlattice_N_Wells_WF2' + ".in")
input_file.save(my_input_file_new, overwrite = True, automkdir = True)

We now create a function to run the input file and ouput the result

In [ ]:
def execute_input_file(input_file, software, biases, datafiles):
    software = 'nextnano++'
    d = {}
    filename_no_extension = get_filename_no_extension(input_file)
    FileExtension = get_filename_extension(input_file)
    
    output_folder = nn.config.config[software]['outputdirectory']
    input_file.execute()
    
    for bias in biases:
            d[bias] = {}
            
            for k,v in datafiles.items():
#                 k = k.replace('00000', bias)
                temp_key = re.search(r'([^\\]+$)', k).group(0)
                if 'bias' in temp_key:
                    d[bias] = {}
                
                    for dat in v:
                        dat = dat.replace('00000', bias)
                        foldername = get_filename_no_extension(input_file)
                        p = os.path.join(output_folder, fr'{foldername}' + fr'\{k}' + fr'\{dat}')
                        data = nn.DataFile(p, product = software)
                        d[bias][dat] = data
                else:
                    d[bias][temp_key] = {}
                
                    for dat in v:
                        dat = dat.replace('00000', bias)
                        p = os.path.join(output_folder, fr'{foldername}' + fr'\{k}' + fr'\{dat}')
                        data = nn.DataFile(p, product = software)
                        d[bias][temp_key][dat] = data
    return d

In [ ]:
df = execute_input_file(input_file, 'nextnano++', ['00000'], datafiles)

In [ ]:
df['00000']['bandedges.dat']

We will take similar steps to the exercise above where we plotted the wavefunction. Because this <code>DataFile</code> does not contain a list of values we swept over, we need not include any $N$ key. That is, we **do not** have to specify <code>df[n]</code>.

In [ ]:
wf2 = get_array(data = df, subfolder = 'Quantum', data_filename = 'amplitudes_shift_quantum_region_Gamma_00000.dat',
               quantity_of_interest = 'Psi_1', func = lambda x : (x-bandedgeoffset) * 1000)
bandedge = get_array(data = df, data_filename = 'bandedges.dat',
                    quantity_of_interest = 'Gamma', func = lambda x : (x-bandedgeoffset) * 1000)

x = get_coords(data = df, data_filename = 'bandedges.dat', direction = 'x', subfolder = None)

xwf2 = get_coords(data = df, data_filename = 'amplitudes_shift_quantum_region_Gamma_00000.dat',
                 subfolder = 'Quantum', bias = '00000', direction = 'x')

**Plotting the figure**

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, bandedge, color = palette[0])
ax.plot(xwf2, wf2, palette[1])

ax.set_ylim([0,450])
ax.set_xlabel('Distance (nm)')
ax.set_ylabel('Energy (meV)')
ax.set_title('Multiple Quantum Well')

# plt.savefig('./Images/wf2.png', dpi = 2000)